In [1]:
"""
2-input XOR example -- this is most likely the simplest possible example.
"""

from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize

from explaneat.core.neuralneat import NeuralNeat as nneat


random.seed(4242)
# random.seed(43)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")

def xor(a, b):
    response = False
    if a > 0.5 and b < 0.5:
        response = True
    if a < 0.5 and b > 0.5:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append(tuple([
            random.uniform(min, max) for ii in range(size)
        ]))

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0


# 2-input XOR inputs and expected outputs.
# xor_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
# xor_outputs = [   (0.0,),     (1.0,),     (1.0,),     (0.0,)]


xs = create_n_points(400, 2)

ys = [
    tuple( [xor(tup[0], tup[1])] ) for tup in xs
]
# def eval_genomes(genomes, config):
    
#     print(genomes)
#     loss = nn.BCELoss()
#     loss = loss.to(device)
#     for genome_id, genome in genomes.items():
#         net = neat.nn.FeedForwardNetwork.create(genome, config)
#         preds = []
#         for xi in xor_inputs:
#             preds.append(net.activate(xi))
#         genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

def eval_genomes(genomes, config):
    for genome_id, genome in genomes.items():
        genome.fitness = 400.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(xs, ys):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2



def run(config_file, runNumber):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)



    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, xs, ys)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)

    # Run for up to 50 generations.
    winner = p.run(eval_genomes, 50, nEpochs = 200)
    # winner = p.run(eval_genomes, 3)
    # winner = p.run(eval_genomes, 10)

    p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
    
    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    # for xi, xo in zip(xor_inputs, xor_outputs):
    #     output = winner_net.activate(xi)
    #     print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    results = []
    for xi, xo in zip(xs, ys):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))
        results.append([xi[0], xi[1], output[0]])

    df = pd.DataFrame(results)
    df.to_csv('results.csv')

    node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.plot_stats(stats, ylog=False, view=False)
    visualize.plot_species(stats, view=False)

    return p

    # p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    # p.run(eval_genomes, 10)



In [2]:

# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
#     local_dir = os.path.dirname(__file__)
# config_path = os.path.join(local_dir, 'config-feedforward')
config_path = os.path.join('./', 'config-backprop')
p = run(config_path, 0)

g = p.best_genome

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

net = nneat(g, config) 

winnerNet = neat.nn.FeedForwardNetwork.create(g, config)

ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

print('have ancestry')

ancestors = {
    k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
}
print('have ancestors')
visualize.create_ancestry_video(p.config, g, ancestry, ancestors, p.reporters.reporters[1])
print('have finished video')

The function - generationStart - has just started at 1635036331.379721

 ****** Running generation 0 ****** 

The function - generationStart - took 0.00013494491577148438 seconds to complete
The function - pre_backprop - has just started at 1635036331.379981
The function - pre_backprop - took 1.4781951904296875e-05 seconds to complete
The function - backprop - has just started at 1635036331.38001
about to start backprop with 200 epochs
mean improvement: -0.05913892962666761
best improvement: -0.1851516109351562
best loss: 0.4968180301384258
The function - backprop - took 3.688030958175659 seconds to complete
The function - post_backprop - has just started at 1635036335.068079
The function - post_backprop - took 4.00543212890625e-05 seconds to complete
The function - evaluate fitness - has just started at 1635036335.068205
The function - evaluate fitness - took 0.09934377670288086 seconds to complete
The function - post evaluate - has just started at 1635036335.1677282
Population's aver

SystemError: <class 'cv2.VideoWriter'> returned a result with an error set

In [1]:
g

NameError: name 'g' is not defined